In [ ]:
import pandas as pd
books_df = pd.read_json('20s_best_book.json')
books_df.head()

In [ ]:
import requests
isbn = 9791190090018
url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
r = requests.get(url.format(isbn))
print(r.text)

In [9]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
prd_link = soup.find('a', attrs={'class':'gd_name'})
print(prd_link)

In [ ]:
print(prd_link['href'])

url = 'http://www.yes24.com' + prd_link['href']
r = requests.get(url)
print(r.text)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
prd_detail = soup.find('div', attrs={'id' : 'infoset_specific'})
print(prd_detail)

In [ ]:
prd_tr_list = prd_detail.find_all('tr')

In [ ]:
for tr in prd_tr_list:
    if tr.find('th').get_text() == '쪽수, 무게, 크기':
        page_td = tr.find('td').get_text()
        break

print(page_td.split()[0])

330쪽 | 496g | 130*198*30mm
330쪽


추가 과제

In [ ]:
def get_page_cnt(isbn):
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    r = requests.get(url.format(isbn))

    soup = BeautifulSoup(r.text, 'html.parser')
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return
    
    url = 'http://www.yes24.com' + prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    prd_detail = soup.find('div', attrs={'id' : 'infoset_specific'})
    prd_tr_list = prd_detail.find_all('tr')

    for tr in prd_tr_list:
        if tr.find('th').get_text() == '쪽수, 무게, 크기':
            return tr.find('td').get_text().split()[0]
    
    return ''

get_page_cnt(9791190090018)
    

'330쪽'

In [25]:
books = pd.read_json('20s_best_book.json')
top10_books = books.head(10)

page_count = top10_books.apply(lambda row: get_page_cnt(row['isbn13']), axis = 1)
page_count.name = 'page_count'
print(page_count)

0    330쪽
1    300쪽
2    228쪽
3    340쪽
4    264쪽
5    272쪽
6    396쪽
7    None
8    376쪽
9    244쪽
Name: page_count, dtype: object


In [28]:
top10_with_page_count = pd.merge(top10_books,page_count, left_index=True, right_index=True)
print(top10_with_page_count.loc[::,'no':'page_count'])

   no  ranking                     bookname                  authors  \
0   1        1  우리가 빛의 속도로 갈 수 없다면 :김초엽 소설                  지은이: 김초엽   
1   2        2         달러구트 꿈 백화점 :이미예 장편소설                 지은이: 이미예   
2   3        3          지구에서 한아뿐 :정세랑 장편소설                  지은이: 정세랑   
3   4        4           시선으로부터, :정세랑 장편소설                  지은이: 정세랑   
4   5        5                  아몬드손원평 장편소설                      손원평   
5   6        6          목소리를 드릴게요 :정세랑 소설집                  지은이: 정세랑   
6   7        7            피프티 피플 :정세랑 장편소설                  지은이: 정세랑   
7   8        8  나미야 잡화점의 기적 :히가시노 게이고 장편소설   지은이: 히가시노 게이고 ;옮긴이: 양윤옥   
8   9        9           천 개의 파랑 :천선란 장편소설                  지은이: 천선란   
9  10       10                   선량한 차별주의자                    김지혜 지음   

  publisher  publication_year         isbn13 addition_symbol vol  class_no  \
0        허블              2019  9791190090018           03810         813.7   
1     팩토리나인              2020  9791165341909       